In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import re
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import boxcox
from scipy.stats import yeojohnson
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Week 2 Briefing
This week, our goal is to learn the basics of statistical tools required for data engineering. There are four modules as part of this week's learning goals:
* Data cleaning
* Statistical queries on dataset
* Data visualization
* Data transformation

# Module 1: Data cleaning
Packages used: pandas, numpy, re, sklearn.impute
Steps: EDA, Missing data handling, removing duplicates, formatting and data type, string cleaning and standardization, outliers, validation, text cleaning.

In [ ]:
# Load dataset
df = pd.read_csv('/kaggle/input/dirty-dataset-to-practice-data-cleaning/my_file (1).csv')
df.head() #preview first few rows to understand the structure


In [ ]:
df.info() #concise summary of the dataset

In [ ]:
df.columns #list the columns

In [ ]:
#Exploratory data analysis: identify missing values, duplicates, data types, summary
print(df.isnull().sum())

In [ ]:
print(df.duplicated())


In [ ]:
print(df.describe(include='all'))


In [ ]:
print(df.dtypes)

In [ ]:
#text cleaning using anonymous function lambda
df["Ref."]= df["Ref."].apply(lambda x : re.sub(r"[^a-zA-Z0-9]" , "" , x))
df

In [ ]:
print([{col} for col in df.columns])

In [ ]:
#remove unwarranted spaces
df.columns = df.columns.str.replace('\xa0', ' ', regex=False)
print([col for col in df.columns])

In [ ]:
#cleaning text
def clean_text(text):
    text = re.sub(r"\[\s*.*?\s*\]", "" , text)
    text = re.sub(r"[^a-zA-Z0-9]", "" , text)
    return text 
columns = ["Actual gross" , "Actual gross(in 2022 dollars)" , "Average gross"]
for column in columns:
    df[column] = df[column].apply(clean_text)
df

In [ ]:
#cleaning texts
def clean_peak(text):
    if not pd.isna(text):
        text =  re.sub(r"\[\s*.*?\s*\]", "" , text)
    return text
columns = ["Peak" , "All Time Peak"]
for column in columns:
    df[column] = df[column].apply(clean_peak)
df

In [ ]:
#remove wierd characters from the strings
def clean_title(text):
    text =  re.sub(r"[^a-zA-Z ]", "" , text)
    return text
df["Tour title"] = df["Tour title"].apply(clean_title)

In [ ]:
df

In [ ]:
#Fixing inconsistent data
df["Ref."] = df["Ref."].replace("1516" , "16")
df["Ref."] = df["Ref."].replace("d" , "5")

In [ ]:
df

In [ ]:
#Fixing numerical columns data type; convert to float
numeric_cols = ["Actual gross" , "Actual gross(in 2022 dollars)" 
                , "Average gross" , "Shows" , "Ref." , "Peak" , "All Time Peak"]
for cols in numeric_cols :
    df[cols] = df[cols].astype("float")

In [ ]:
df

In [ ]:
#drop columns with too many missing values
df = df.drop(["All Time Peak" , "Peak"] , axis =1)
df

In [ ]:
#Final validation
print(df.isnull().sum())
print(df.describe(include='all'))
print(df.head())

# Save cleaned version
df.to_csv('dirty_dataset_cleaned.csv', index=False)


# Module 2: Querying data, statistical properties and error
Packages used: pandas

In [ ]:
# Load data 
df = pd.read_csv("/kaggle/input/students-performance-in-exams/StudentsPerformance.csv")
df.head()

In [ ]:
#Exploration: looking for numerical data
numeric_cols = ['math score', 'reading score', 'writing score']
df[numeric_cols].describe()

#Exercise 1: Add a new column that computes average score of a student across the three subjects

In [ ]:
# Exercise 1

df = pd.read_csv("/kaggle/input/students-performance-in-exams/StudentsPerformance.csv")

df['average score'] = df[['math score', 'reading score', 'writing score']].mean(axis=1)
df.head()


In [ ]:
# Mean, mode and median
print("Mean Math Score:", df['math score'].mean())
print("Median Math Score:", df['math score'].median())
print("Mode Math Score:", df['math score'].mode()[0])

#Exercise 2: Compute mean, median, mode for reading and writing scores.

In [ ]:
# Exercise 2: 

# Reading
print("Mean Reading Score:", df['reading score'].mean())
print("Median Reading Score:", df['reading score'].median())
print("Mode Reading Score:", df['reading score'].mode()[0])

# Writing
print("Mean Writing Score:", df['writing score'].mean())
print("Median Writing Score:", df['writing score'].median())
print("Mode Writing Score:", df['writing score'].mode()[0])


In [ ]:
df['math score'][1]

In [ ]:
# Measuring spread of data
print("Standard Deviation (Math):", df['math score'].std())
print("Variance (Math):", df['math score'].var())
print("Range (Math):", df['math score'].max() - df['math score'].min())

#Exercise 3: Repeat above steps for other subjects. What do you infer about subject difficulty?

In [ ]:
# Exercise 3:

# Reading
print("Std Dev (Reading):", df['reading score'].std())
print("Variance (Reading):", df['reading score'].var())
print("Range (Reading):", df['reading score'].max() - df['reading score'].min())

# Writing
print("Std Dev (Writing):", df['writing score'].std())
print("Variance (Writing):", df['writing score'].var())
print("Range (Writing):", df['writing score'].max() - df['writing score'].min())

# Most difficult subject (highest std deviation)
std_dict = {
    'Math': df['math score'].std(),
    'Reading': df['reading score'].std(),
    'Writing': df['writing score'].std()
}

most_difficult = max(std_dict, key=std_dict.get)
print("Most Difficult Subject:", most_difficult)


# Module 3: Data visualization 
Packages used: seaborn, matplotlib 

In [ ]:
sns.histplot(df['math score'], kde=True)
plt.axvline(df['math score'].mean(), color='red', linestyle='--', label='Mean')
plt.axvline(df['math score'].median(), color='green', linestyle='--', label='Median')
plt.legend()
plt.title("Distribution of Math Scores")
plt.show()

#Exercise 4: Repeat for reading and writing scores

In [ ]:
# Exercise 4: 
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x='test preparation course', y='math score', hue='gender', data=df)
plt.title("Math Scores by Test Prep & Gender", fontsize=14)
plt.xlabel("Test Preparation Course")
plt.ylabel("Math Score")
plt.legend(title="Gender", loc="upper right")
plt.show()


In [ ]:
#Comarisons using visualization
sns.boxplot(x='test preparation course', y='math score', data=df)
plt.title("Math Score vs Test Preparation")

#Exercise 5: Try changing legends, title, axis labels

In [ ]:
# Exercise 5: Plot distributions other than Gaussian and Poisson
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom, expon

# Binomial Distribution
x = np.arange(0, 21)
p = 0.5
binomial_pmf = binom.pmf(x, n=20, p=p)
plt.bar(x, binomial_pmf)
plt.title("Binomial Distribution (n=20, p=0.5)")
plt.show()

# Exponential Distribution
x_exp = np.linspace(0, 10, 100)
exp_pdf = expon.pdf(x_exp, scale=1)
plt.plot(x_exp, exp_pdf, color='green')
plt.title("Exponential Distribution (λ=1)")
plt.show()



In [ ]:
sns.displot(df['math score'], kde=True)

## Module 3.1: Generating a normal distribution 
* np.random.normal(loc=mu, scale=sigma, size=n) generates random numbers from a normal (Gaussian) distribution.
* plt.hist(..., density=True) makes the histogram show a probability density, not just raw counts.
* The red line overlays the theoretical probability density function (PDF) for comparison.

In [ ]:
# Parameters for the normal distribution
mu = 0       # Mean
sigma = 1    # Standard deviation
n = 10000    # Number of samples

# Generate random samples from a normal distribution
data = np.random.normal(loc=mu, scale=sigma, size=n)

# Plot histogram of the data
plt.figure(figsize=(8, 5))
plt.hist(data, bins=50, density=True, alpha=0.6, color='skyblue', edgecolor='black')

# Plot the theoretical normal distribution curve
from scipy.stats import norm
x = np.linspace(mu - 4*sigma, mu + 4*sigma, 1000)
plt.plot(x, norm.pdf(x, mu, sigma), 'r', label='Theoretical PDF')

plt.title("Generated Normal Distribution (μ=0, σ=1)")
plt.xlabel("Value")
plt.ylabel("Probability Density")
plt.legend()
plt.grid(True)
plt.show()


## Module 3.2: Generating a theoretical poisson/gaussian distribution

In [ ]:
#Poisson distribution
from scipy.stats import poisson

x = np.arange(0, 20)
pmf = poisson.pmf(x, mu=5)
plt.bar(x, pmf)
plt.title("Poisson Distribution (μ=5)")

In [ ]:
#Gaussian distribution
pdf=norm.pdf(x,mu,sigma)
plt.bar(x,pdf)
plt.title("Gaussian distribution")

# Module 4: Data transformations - demonstrating Log transformation
Packages used: matplotlib, seaborn, scipy.stats

Dataset: heart-disease-uci/heart.csv

In [ ]:
#Loading data
dc=pd.read_csv("/kaggle/input/heart-disease-data/heart_disease_uci.csv")

#Plotting the original cholestrol distribution data
sns.displot(dc["chol"],kde=True)
plt.title("DISTRIBUTION OF CHOLESTROL LEVEL",fontsize=20)
skewness=str(dc["chol"].skew()) #measures the skewness
kurtosis=str(dc["chol"].kurt())
plt.legend([skewness,kurtosis],title=("skewness and kurtosis"))
plt.show()

# Assignment Exercises

* Complete the six exercises given above as comments in various code blocks.
* Repeat (all or most or as appropriate) the above workbook steps for data engineering and visualization for the 'students-random-number-dataset' already linked to this notebook.
* Plot the distributions discussed in class, other than gaussian and poisson.
* For the dataset in CELL 30-31 above, apply other transformations (not the Log transform) discussed in class.

Submission steps: 
* Clone and Edit this notebook to add all new exercises below this cell.
* Once finished, connect this notebook to github.
* Using the assignment link, join the github classroom by connecting your github account with already available name roster containing your institute email.
* Commit!

In [ ]:
#Log transformation
log_target=np.log1p(dc["chol"])
ax=sns.distplot(log_target)
plt.title("DISTRIBUTION AFTER LOG TRANSFORMATION",)
skewness=str(log_target.skew())
plt.legend([skewness],title=("skewness"))
plt.show()

# Exercise 6: Repeat this for square root and other transformations.

In [ ]:
# Exercise 6: 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import yeojohnson

# Load dataset
dc = pd.read_csv("/kaggle/input/heart-disease-data/heart_disease_uci.csv")

# Keep only finite numbers for transformations
chol_clean = dc["chol"].replace([np.inf, -np.inf], np.nan).dropna()

# Square root transformation
sqrt_target = np.sqrt(chol_clean)
sns.histplot(sqrt_target, kde=True)
plt.title("Square Root Transformation")
plt.show()

# Yeo-Johnson transformation
yj_target, _ = yeojohnson(chol_clean)
sns.histplot(yj_target, kde=True)
plt.title("Yeo-Johnson Transformation")
plt.show()


In [ ]:
# Square Root Transformation
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
dc = pd.read_csv("/kaggle/input/heart-disease-data/heart_disease_uci.csv")

# Clean data: remove NaN and infinite values
chol_clean = dc["chol"].replace([np.inf, -np.inf], np.nan).dropna()

# Apply square root transformation
sqrt_target = np.sqrt(chol_clean)

# Plot distribution
sns.histplot(sqrt_target, kde=True)
plt.title("Square Root Transformation of Cholesterol Levels")
plt.xlabel("Transformed Cholesterol")
plt.ylabel("Frequency")
plt.show()


In [ ]:
# Load dataset
df = pd.read_csv('/kaggle/input/students-random-number-dataset/stud-randnum-dataset.csv')
df.head() #preview first few rows to understand the structure


In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
print(df.isnull().sum())

In [ ]:
print(df.duplicated())

In [ ]:
print(df.describe(include='all'))

In [ ]:
print(df.dtypes)

In [ ]:
# Square Root Transformation
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
dc = pd.read_csv("/kaggle/input/heart-disease-data/heart_disease_uci.csv")

# Clean data: remove NaN and infinite values
chol_clean = dc["chol"].replace([np.inf, -np.inf], np.nan).dropna()

# Apply square root transformation
sqrt_target = np.sqrt(chol_clean)

# Plot distribution
sns.histplot(sqrt_target, kde=True)
plt.title("Square Root Transformation of Cholesterol Levels")
plt.xlabel("Transformed Cholesterol")
plt.ylabel("Frequency")
plt.show()


In [ ]:
# Square Root Transformation for dataset in Cell 30-31
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
dc = pd.read_csv("/kaggle/input/heart-disease-data/heart_disease_uci.csv")

# Clean the 'chol' column: remove NaN and infinite values
chol_clean = dc["chol"].replace([np.inf, -np.inf], np.nan).dropna()

# Apply square root transformation
sqrt_target = np.sqrt(chol_clean)

# Plot
sns.histplot(sqrt_target, kde=True)
plt.title("Square Root Transformation of Cholesterol Levels")
plt.xlabel("Transformed Cholesterol")
plt.ylabel("Frequency")
plt.show()
